In [28]:
import pmdarima as pm
import pandas as pd
import numpy as np
from pandas import datetime
from math import sqrt

import yfinance as yf
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from catboost import CatBoostRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
import os
from joblib import dump, load

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot
import seaborn as sns

In [29]:
tickers=['BRK-A']

df = yf.download(tickers=tickers,start="2010-05-04", end = "2022-05-10", interval='1d')

[*********************100%***********************]  1 of 1 completed


In [30]:
for i in range(0,len(df.columns)):
    if (df == 0).any()[i]==True:
        df = df.drop(df.columns[i], axis=1)
df_x=df.iloc[:, 0:len(df.columns)].dropna()
i=len(df_x)

while i>0:
    df_x.iloc[i-1]=df_x.iloc[i-2]
    i=i-1
df_total=df_x
df_total=df_total.dropna()
l=20
train=df_total[:-l]
test=df_total[len(train):]

In [31]:
from pmdarima import auto_arima
import pmdarima as pm

In [32]:
# Fit a simple auto_arima model
arima = pm.auto_arima(train['Close'],
                      seasonal=True,
                      stationary=False,
                      trace=True,
                      )
pmd_predictions = arima.predict(n_periods=len(test['Close'][-l:]))
arima_mae = np.mean(np.abs(test['Close'][-l:] - pmd_predictions))
arima_rmse = (np.mean((test['Close'][-l:] - pmd_predictions)**2))**.5
arima_mape = np.sum(np.abs(pmd_predictions - test['Close'][-l:])) / (np.sum((np.abs(test['Close'][-l:]))))

Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=inf, Time=1.44 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=62502.253, Time=0.04 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=59349.677, Time=0.10 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.30 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=58186.355, Time=0.15 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=57721.456, Time=0.29 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=57527.372, Time=0.36 sec
 ARIMA(5,2,0)(0,0,0)[0]             : AIC=57381.434, Time=0.56 sec
 ARIMA(5,2,1)(0,0,0)[0]             : AIC=inf, Time=2.36 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=inf, Time=2.86 sec
 ARIMA(5,2,0)(0,0,0)[0] intercept   : AIC=57383.426, Time=1.06 sec

Best model:  ARIMA(5,2,0)(0,0,0)[0]          
Total fit time: 9.542 seconds


In [33]:
pmd_predictions=pd.DataFrame(pmd_predictions)

In [34]:
predictions = pmd_predictions.set_index((pd.DataFrame(test['Close']).index))
result = pd.concat([predictions, pd.DataFrame(pd.DataFrame(test['Close']))], axis=1)
result.rename(columns = {0:'pred', 'Close':'y'}, inplace = True)

In [35]:
fig = px.line(result, y=result.columns[0:2], x=result.index)
fig.show()

In [36]:
MAE = mean_absolute_error(result['y'], result['pred'])
RSQUARED = r2_score(result['y'], result['pred'])
MAPE = mean_absolute_percentage_error(result['y'], result['pred'])
MSE = mean_squared_error(result['y'], result['pred'])
RMSE = math.sqrt(MSE)
COLUMN_NAMES = ["MSE", "RMSE", "MAE", "MAPE", "RSQUARED"]
values = np.array([f'{MSE:.2f}', f'{RMSE:.2f}', f'{MAE:.2f}', f'{MAPE:.2f}', f'{RSQUARED:.2f}'])

df = pd.DataFrame(values, columns=['Results'], index=COLUMN_NAMES)
df

,Results
MSE,246766635.71
RMSE,15708.81
MAE,12996.80
MAPE,0.03
RSQUARED,0.17


In [37]:
from ThymeBoost import ThymeBoost as tb

In [38]:
#get the order
auto_order = arima.get_params()['order']
boosted_model = tb.ThymeBoost(verbose=1)
output = boosted_model.fit(train['Close'],
                            trend_estimator='arima',
                            arima_order=auto_order,
                            global_cost='mse')
predicted_output = boosted_model.predict(output, len(test['Close']))
tb_mae = np.mean(np.abs(test['Close'] - predicted_output['predictions']))
tb_rmse = (np.mean((test['Close'] - predicted_output['predictions'])**2))**.5
tb_mape = np.sum(np.abs(predicted_output['predictions'] - test['Close'])) / (np.sum((np.abs(test['Close']))))

********** Round 1 **********
Using Split: None
Fitting initial trend globally with trend model:
median()
seasonal model:
None
cost: 9669082585.779182
********** Round 2 **********
Using Split: None
Fitting global with trend model:
arima((5, 2, 0))
seasonal model:
None
cost: 113787711.60644749
Boosting Terminated 
Using round 2


In [39]:
predictions=pd.DataFrame(predicted_output['predictions'])
predictions = predictions.set_index((pd.DataFrame(test['Close']).index))
result = pd.concat([predictions, pd.DataFrame(pd.DataFrame(test['Close']))], axis=1)
result.rename(columns = {'predictions':'pred', 'Close':'y'}, inplace = True)

In [40]:
fig = px.line(result, y=result.columns[0:2], x=result.index)
fig.show()

In [41]:
MAE = mean_absolute_error(result['y'], result['pred'])
RSQUARED = r2_score(result['y'], result['pred'])
MAPE = mean_absolute_percentage_error(result['y'], result['pred'])
MSE = mean_squared_error(result['y'], result['pred'])
RMSE = math.sqrt(MSE)
COLUMN_NAMES = ["MSE", "RMSE", "MAE", "MAPE", "RSQUARED"]
values = np.array([f'{MSE:.2f}', f'{RMSE:.2f}', f'{MAE:.2f}', f'{MAPE:.2f}', f'{RSQUARED:.2f}'])

df = pd.DataFrame(values, columns=['Results'], index=COLUMN_NAMES)
df

,Results
MSE,148746873.77
RMSE,12196.18
MAE,10570.87
MAPE,0.02
RSQUARED,0.50


In [42]:
#Additional part with FB Prophet

In [43]:
df_total['datetime']=df_total.index

In [44]:
from fbprophet import Prophet

In [75]:
df_total

,Open,High,Low,Close,Adj Close,Volume,datetime,cap,floor
Date,,,,,,,,,
2010-05-03,482864.0,483795.0,475000.0,479500.0,479500.0,25,2010-05-03,483795.0,475000.0
2010-05-04,116190.0,117950.0,116190.0,117430.0,117430.0,591,2010-05-04,117950.0,116190.0
2010-05-05,117100.0,117100.0,114499.0,114800.0,114800.0,606,2010-05-05,117100.0,114499.0
2010-05-06,113500.0,115365.0,113150.0,114950.0,114950.0,905,2010-05-06,115365.0,113150.0
2010-05-07,114543.0,115726.0,108565.0,113500.0,113500.0,1391,2010-05-07,115726.0,108565.0
...,...,...,...,...,...,...,...,...,...
2022-05-03,485921.0,492125.0,470692.0,478150.0,478150.0,41,2022-05-03,492125.0,470692.0
2022-05-04,480541.0,487713.0,478150.0,481198.0,481198.0,24,2022-05-04,487713.0,478150.0
2022-05-05,486664.0,494343.0,477712.0,494343.0,494343.0,24,2022-05-05,494343.0,477712.0


In [96]:
l=20
# cap = df_total['High']
# flr = df_total['Low']
# df_total['cap'] = cap
# df_total['floor'] = flr
temp_df = df_total.reset_index()
temp_df = temp_df[['datetime', 'Close', 'cap', 'floor']]
temp_df.rename(columns={'datetime': 'ds', 'Close': 'y'}, inplace=True)
train, test = temp_df.iloc[:-l,:], temp_df.iloc[-l:,:]
m = Prophet(
            growth='linear',
            seasonality_mode='additive',
            interval_width=1,
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=True
            )
m.fit(train)
predictions_train = m.predict(train.drop('y', axis=1))
predictions_test = m.predict(test.drop('y', axis=1))
result = pd.concat([predictions_train, predictions_test], axis=0)
result=result[-20:]
result.index=result['ds']
result=result.drop(['ds'],axis=1)
result=pd.concat([result, df_total[-20:]['Close']],axis=1)
result.rename(columns = {'yhat':'pred', 'Close':'y'}, inplace = True)

In [97]:
fig = px.line(result, y=result.columns[0:len(result.columns)], x=result.index)
fig.show()

In [98]:
MAE = mean_absolute_error(result['y'], result['pred'])
RSQUARED = r2_score(result['y'], result['pred'])
MAPE = mean_absolute_percentage_error(result['y'], result['pred'])
MSE = mean_squared_error(result['y'], result['pred'])
RMSE = math.sqrt(MSE)
COLUMN_NAMES = ["MSE", "RMSE", "MAE", "MAPE", "RSQUARED"]
values = np.array([f'{MSE:.2f}', f'{RMSE:.2f}', f'{MAE:.2f}', f'{MAPE:.2f}', f'{RSQUARED:.2f}'])

df = pd.DataFrame(values, columns=['Results'], index=COLUMN_NAMES)
df

,Results
MSE,1327126474.82
RMSE,36429.75
MAE,30704.52
MAPE,0.06
RSQUARED,-3.44


In [159]:
future= m.make_future_dataframe(periods=20)

In [160]:
result=m.predict(future)[(len(result))-20:]
result

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
20,2010-06-01,409200.506952,61344.978216,170640.313899,409200.506952,409200.506952,-288572.692712,-288572.692712,-288572.692712,-270559.200643,...,-13651.386141,-13651.386141,-13651.386141,-4362.105928,-4362.105928,-4362.105928,0.0,0.0,0.0,120627.814240
21,2010-06-02,409177.247111,56435.709982,184090.494165,409177.247111,409177.247111,-288638.089872,-288638.089872,-288638.089872,-270559.200643,...,-13774.983482,-13774.983482,-13774.983482,-4303.905747,-4303.905747,-4303.905747,0.0,0.0,0.0,120539.157239
22,2010-06-03,409153.987270,64811.139915,182018.845307,409153.987270,409153.987270,-288460.264837,-288460.264837,-288460.264837,-270559.200643,...,-13660.729270,-13660.729270,-13660.729270,-4240.334923,-4240.334923,-4240.334923,0.0,0.0,0.0,120693.722433
23,2010-06-04,409130.727428,64077.380615,183651.782045,409130.727428,409130.727428,-288053.104509,-288053.104509,-288053.104509,-270559.200643,...,-13317.632944,-13317.632944,-13317.632944,-4176.270922,-4176.270922,-4176.270922,0.0,0.0,0.0,121077.622919
24,2010-06-07,409060.947905,62050.458087,173672.625452,409060.947905,409060.947905,-287821.053028,-287821.053028,-287821.053028,-270559.200643,...,-13235.068161,-13235.068161,-13235.068161,-4026.784223,-4026.784223,-4026.784223,0.0,0.0,0.0,121239.894877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3022,2022-04-24,756695.465165,450855.288461,581757.813975,755330.339131,759097.484195,-234685.006519,-234685.006519,-234685.006519,-270559.200643,...,33819.899702,33819.899702,33819.899702,2054.294422,2054.294422,2054.294422,0.0,0.0,0.0,522010.458646
3023,2022-04-25,756911.849713,421176.204371,536186.737785,755410.822703,759485.761338,-281571.185640,-281571.185640,-281571.185640,-270559.200643,...,-13235.068161,-13235.068161,-13235.068161,2223.083165,2223.083165,2223.083165,0.0,0.0,0.0,475340.664073
3024,2022-04-26,757128.234261,422633.979025,533806.716199,755491.306274,759874.038480,-281838.728801,-281838.728801,-281838.728801,-270559.200643,...,-13651.386141,-13651.386141,-13651.386141,2371.857982,2371.857982,2371.857982,0.0,0.0,0.0,475289.505459
3025,2022-04-27,757344.618809,412426.462980,526144.725118,755571.789845,760262.315623,-281838.895309,-281838.895309,-281838.895309,-270559.200643,...,-13774.983482,-13774.983482,-13774.983482,2495.288816,2495.288816,2495.288816,0.0,0.0,0.0,475505.723499


In [161]:
tickers=['BRK-A']

df = yf.download(tickers=tickers,start="2010-05-04", end = "2022-04-29", interval='1d')

[*********************100%***********************]  1 of 1 completed


In [162]:
df=df['Close'][-20:]

In [165]:
result=m.predict(future)
result=result[-20:]

In [166]:
df.index=result.index

In [167]:
result=pd.concat([result, df], axis=1)

In [168]:
fig = px.line(result, y=result.columns[1:len(result.columns)], x=result.index)
fig.show()

In [171]:
result.rename(columns = {'yhat':'pred', 'Close':'y'}, inplace = True)

In [172]:
MAE = mean_absolute_error(result['y'], result['pred'])
RSQUARED = r2_score(result['y'], result['pred'])
MAPE = mean_absolute_percentage_error(result['y'], result['pred'])
MSE = mean_squared_error(result['y'], result['pred'])
RMSE = math.sqrt(MSE)
COLUMN_NAMES = ["MSE", "RMSE", "MAE", "MAPE", "RSQUARED"]
values = np.array([f'{MSE:.2f}', f'{RMSE:.2f}', f'{MAE:.2f}', f'{MAPE:.2f}', f'{RSQUARED:.2f}'])

df = pd.DataFrame(values, columns=['Results'], index=COLUMN_NAMES)
df

,Results
MSE,1328553934.85
RMSE,36449.33
MAE,31939.57
MAPE,0.06
RSQUARED,-12.17
